In [0]:
import random
import time
import json
from faker import Faker
from confluent_kafka import Producer
from faker.providers import BaseProvider
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField

In [0]:
def read_config():
  config = {}
  with open("/dbfs/FileStore/client_properties") as fh:
    for line in fh:
      line = line.strip()
      if len(line) != 0 and line[0] != "#":
        parameter, value = line.strip().split('=', 1)
        config[parameter] = value.strip()
  return config

conf = read_config()

In [0]:
from confluent_kafka.schema_registry import SchemaRegistryClient

topic = 'mobile_client'

# cambiar por las vuestras
schema_registry_url = "https://psrc-ko92v.us-east-2.aws.confluent.cloud"
schema_registry_username = 'G3YDZKKPA2MGGLQD'
schema_registry_password = 'PbYRQGwmGdntB9bcpFh4CHeBv9B7+MWA2YM/UETXYKLiFCOqoLVR6xq/wNq9/UOo'
#schema_registry_username = "DRPEYI5PVVYELBEL"
#schema_registry_password = "rgsuii1ahABRHD8GXqFKkNSF4vJqXZcv/nefDdaJmW3ud9euF0vwZPzfvV0zABw1"

schema_registry_conf = {'url': schema_registry_url,
                        'basic.auth.user.info' : f'{schema_registry_username}:{schema_registry_password}'}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [0]:
import random
import time
import json
from faker import Faker
from confluent_kafka import Producer
from faker.providers import BaseProvider
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer

class MobileUserEventProvider(BaseProvider):
    def user_id(self):
        return random.randint(1000, 9999)

    def event_type(self):
        event_types = ["login", "logout", "purchase", "view", "click"]
        return random.choice(event_types)

    def location(self):
        locations = ["Farm A", "Farm B", "Farm C", "Farm D"]
        return random.choice(locations)

    def produce_msg(self, FakerInstance, event_count=1):
        events = []
        for _ in range(event_count):
            event = {
                "userId": self.user_id(),
                "eventType": self.event_type(),
                "location": self.location(),
                "timestamp": int(time.time() * 1000),
            }
            events.append(event)
        return events

# Update Avro schema for mobile user events
event_schema_str = """
{
  "type": "record",
  "name": "UserEvent",
  "fields": [
    {
      "name": "userId",
      "type": "int"
    },
    {
      "name": "eventType",
      "type": "string"
    },
    {
      "name": "location",
      "type": "string"
    },
    {
      "name": "timestamp",
      "type": "long"
    }
  ]
}
"""
# key serializer
string_serializer = StringSerializer('utf_8')

# Update value serializer
avro_serializer = AvroSerializer(schema_registry_client, event_schema_str)

producer = Producer(conf)

fake = Faker()
fake.add_provider(MobileUserEventProvider)

MAX_EVENTS = 10

for i in range(MAX_EVENTS):
    events = fake.produce_msg(fake, 1)
    for event in events:
        producer.produce(topic=topic, 
                         key=string_serializer(str(event["userId"])), 
                         value=avro_serializer(event, SerializationContext(topic, MessageField.VALUE)))
    producer.flush()
    print(f"Event {i+1} sent")
    # sleeping time
    sleep_time = (
        random.randint(0, int(1 * 10000)) / 10000
    )
    print("sleeping for..." + str(sleep_time) + "s")
    time.sleep(sleep_time)